In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

import joblib

from data_preprocessing import pipeline

### 데이터 전처리

In [2]:
# 로우 데이터 불러오기
file_path = "./storage/raw_data/병합_청약매물_목록_정보_픽스.csv"
df = pd.read_csv(file_path, encoding='cp949')

# 데이터 전처리
preprocessing_pipeline =pipeline()
df = preprocessing_pipeline.transform(df)

# 파일 저장
file_version = "250317-01"
output_file = f"./storage/train_data/train-{file_version}.csv"
df.to_csv(output_file, index=False, encoding='cp949')

C:\Users\user\AppData\Local\Temp\ipykernel_9848\475702578.py:3: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='cp949')
c:\Users\user\developments\hk-project\src\data_preprocessing.py:136: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_estate_price = pd.read_csv(


In [71]:
df

,공고번호,주택명,공급지역코드,공급지역명,공급위치우편번호,공급위치,공급규모,모집공고일,청약접수시작일,청약접수종료일,...,미달여부,공급금액(최고가 기준),위도,경도,행정동코드,법정동코드,시도,시군구,읍면동1,읍면동2
0,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,Y,377000000.0,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동
1,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,N,377000000.0,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동
2,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,Y,377000000.0,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동
3,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,Y,377000000.0,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동
4,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,Y,377000000.0,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17245,2020000298,시흥장현 영무예다음,410,경기,14996,경기도 시흥장현 공공주택지구 B-9BL,747,2020-03-20,2020-03-30,2020-04-01,...,N,427200000.0,37.380161,126.790527,4.139063e+09,4.139013e+09,경기,시흥시,장곡동,장곡동
17246,2020000298,시흥장현 영무예다음,410,경기,14996,경기도 시흥장현 공공주택지구 B-9BL,747,2020-03-20,2020-03-30,2020-04-01,...,N,427200000.0,37.380161,126.790527,4.139063e+09,4.139013e+09,경기,시흥시,장곡동,장곡동
17247,2020000298,시흥장현 영무예다음,410,경기,14996,경기도 시흥장현 공공주택지구 B-9BL,747,2020-03-20,2020-03-30,2020-04-01,...,Y,427200000.0,37.380161,126.790527,4.139063e+09,4.139013e+09,경기,시흥시,장곡동,장곡동
17248,2020000298,시흥장현 영무예다음,410,경기,14996,경기도 시흥장현 공공주택지구 B-9BL,747,2020-03-20,2020-03-30,2020-04-01,...,Y,427200000.0,37.380161,126.790527,4.139063e+09,4.139013e+09,경기,시흥시,장곡동,장곡동


### 모델 학습 및 평가

In [67]:
# 전처리된 데이터 가져오기

file_version = "250317-01"
file_path = f"./storage/train_data/train-{file_version}.csv"
df = pd.read_csv(file_path, encoding='cp949')

In [68]:
df

,공고번호,주택명,공급지역코드,공급지역명,공급위치우편번호,공급위치,공급규모,모집공고일,청약접수시작일,청약접수종료일,...,위도,경도,행정동코드,법정동코드,시도,시군구,읍면동1,읍면동2,전용면적당 공급금액(최고가기준),전용면적당 시세차익
0,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동,8.767442e+06,NaN
1,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동,8.767442e+06,NaN
2,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동,8.767442e+06,NaN
3,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동,8.767442e+06,NaN
4,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동,8.767442e+06,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17245,2020000298,시흥장현 영무예다음,410,경기,14996,경기도 시흥장현 공공주택지구 B-9BL,747,2020-03-20,2020-03-30,2020-04-01,...,37.380161,126.790527,4.139063e+09,4.139013e+09,경기,시흥시,장곡동,장곡동,5.085714e+06,5.085392e+06
17246,2020000298,시흥장현 영무예다음,410,경기,14996,경기도 시흥장현 공공주택지구 B-9BL,747,2020-03-20,2020-03-30,2020-04-01,...,37.380161,126.790527,4.139063e+09,4.139013e+09,경기,시흥시,장곡동,장곡동,5.085714e+06,5.085392e+06
17247,2020000298,시흥장현 영무예다음,410,경기,14996,경기도 시흥장현 공공주택지구 B-9BL,747,2020-03-20,2020-03-30,2020-04-01,...,37.380161,126.790527,4.139063e+09,4.139013e+09,경기,시흥시,장곡동,장곡동,5.085714e+06,5.085392e+06
17248,2020000298,시흥장현 영무예다음,410,경기,14996,경기도 시흥장현 공공주택지구 B-9BL,747,2020-03-20,2020-03-30,2020-04-01,...,37.380161,126.790527,4.139063e+09,4.139013e+09,경기,시흥시,장곡동,장곡동,5.085714e+06,5.085392e+06


In [6]:
# TODO: 여러 모델 학습 및 하이퍼파라미터 최적화

model = RandomForestRegressor(
    n_estimators=100,
    max_depth=None,
    random_state=42
)

# model.fit(x, y)

### 모델 저장 

In [7]:
# 모델 저장 및 로드

version = '0.0.1'

# 모델 저장
joblib.dump(model, f"./storage/trained_model/model_{version}.pkl")

['./storage/trained_model/model_0.0.1.pkl']

In [8]:
# 모델 로드
loaded_model = joblib.load("./storage/trained_model/model_0.0.1.pkl")

# 예측
# X_test = np.array([[1, 2]])
# prediction = loaded_model.predict(X_test)
# print("Prediction:", prediction)